In [90]:
import numpy as np
import pandas as pd

#Feb records
data1 = pd.read_csv("rental_data.csv")

#March records
data2 = pd.read_csv("rental_data_March.csv")

data = pd.concat([data1, data2], axis=0)

# check null value count
data.isna().sum()

Post_ID          0
Time             0
House_type       0
Area             0
Area_Name        0
Address       7950
Rent           187
url              0
dtype: int64

In [91]:
data = data.dropna(subset=['Rent'])

data = data.fillna('Not Available')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49951 entries, 0 to 40617
Data columns (total 8 columns):
Post_ID       49951 non-null int64
Time          49951 non-null object
House_type    49951 non-null object
Area          49951 non-null object
Area_Name     49951 non-null object
Address       49951 non-null object
Rent          49951 non-null object
url           49951 non-null object
dtypes: int64(1), object(7)
memory usage: 3.4+ MB


In [92]:
data['Time'] = pd.to_datetime(data['Time'])
data['Rent'] = data.Rent.str.replace(',', '')
data['Rent'] = data['Rent'].str.replace('$','').astype(float)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49951 entries, 0 to 40617
Data columns (total 8 columns):
Post_ID       49951 non-null int64
Time          49951 non-null datetime64[ns]
House_type    49951 non-null object
Area          49951 non-null object
Area_Name     49951 non-null object
Address       49951 non-null object
Rent          49951 non-null float64
url           49951 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 3.4+ MB


In [93]:
samp_series = data["House_type"]

In [94]:
br = []
area_sqft = []
for val in samp_series:
    if 'br' in val:
        br.append(int(val[:val.index('br')]))
        val = val[val.index('br')+3:]
    else:
        br.append(None)
    if 'ft' in val:
        area_sqft.append(int(val[:val.index('ft')]))
    else:
        area_sqft.append(None)

In [95]:
data["bed_rooms"] = br
data["house_size_sqft"] = area_sqft

data.head(2)

,Post_ID,Time,House_type,Area,Area_Name,Address,Rent,url,bed_rooms,house_size_sqft
0,6827734213,2019-02-25 08:07:00,1br-540ft2-,V2A,Penticton,Not Available,850.0,https://kelowna.craigslist.org/apa/d/penticton...,1.0,540.0
1,6826412736,2019-02-23 10:11:00,2br-,V2A,Penticton,Not Available,1100.0,https://kelowna.craigslist.org/apa/d/penticton...,2.0,NaN


In [96]:
data = data.drop_duplicates(subset=['House_type', 'Area','Area_Name','Rent'])

In [97]:
data = data.drop_duplicates(subset='Post_ID')

data = data[["Post_ID","Time","Area","Area_Name","Address","bed_rooms","house_size_sqft","Rent"]]

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13878 entries, 0 to 40614
Data columns (total 8 columns):
Post_ID            13878 non-null int64
Time               13878 non-null datetime64[ns]
Area               13878 non-null object
Area_Name          13878 non-null object
Address            13878 non-null object
bed_rooms          13194 non-null float64
house_size_sqft    10920 non-null float64
Rent               13878 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 975.8+ KB


In [98]:
#data.to_csv("rental_data_cleaned.csv",index=False)
#data.to_csv("rental_data_cleaned_March.csv",index=False)
data.to_csv("all___rental_data_cleaned.csv",index=False)

In [8]:
model_data = data[data['house_size_sqft'] != 0]
model_data = model_data[["Area","bed_rooms","house_size_sqft","Rent"]]

In [10]:
school_data = pd.read_csv("elementary_schools_address.csv")
school_data.head()

,Unnamed: 0,2017-18 rank,rank in the most recent 5 years,trend,School Name,City,2017-18 rating,rating in the most recent 5 years,All Schools,Address
0,0,1/955,1/805,,Cedardale,West Vancouver,10.0,10.0,Add to compare,"595 Burley Dr, West Vancouver, BC V7T 1Z3"
1,1,1/955,1/805,NaN,Corpus Christi,Vancouver,10.0,10.0,Add to compare,NaN
2,2,1/955,1/805,,Crofton House,Vancouver,10.0,10.0,Add to compare,"3200 W 41st Ave, Vancouver, BC V6N 3E1"
3,3,1/955,1/805,,Diamond,Surrey,10.0,10.0,Add to compare,"18620 56 Ave, Surrey, BC V3S 1G1"
4,4,1/955,1/805,,Holy Cross,Burnaby,10.0,10.0,Add to compare,"1450 Delta Ave, Burnaby, BC V5B 3G2"


In [53]:
import re

ar_code = []
school_data['Address'] = school_data['Address'].fillna('unknown')

for val in school_data['Address']:
    if re.search("[V][0-9][ABCEGHJKLMNPRSTVWXYZ]", val , re.IGNORECASE):
        ar_code.append(re.search("[V][0-9][ABCEGHJKLMNPRSTVWXYZ]", val , re.IGNORECASE | re.DOTALL)[0])
    else:
        ar_code.append("none")

school_data["postal_area"] = ar_code
school_data

p_df = school_data.groupby('postal_area').median()
join_df = pd.DataFrame()
join_df['Area'] = p_df['2017-18 rating'].index
join_df['Rating'] = p_df['2017-18 rating'].values
join_df

model_data['Area']= model_data['Area'].astype(str)
join_df['Area']= join_df['Area'].astype(str)


df_inner = pd.merge(model_data, join_df, on='Area', how='inner')
df_inner.rename(columns={'Rating': 'Median_school_rating',}, inplace=True)
fin_df = df_inner[['Area','bed_rooms','house_size_sqft','Median_school_rating','Rent']]

In [54]:
fin_df.to_csv("model_rental_data.csv",index=False)